In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords

### V3

In [2]:
movies_df_raw = pd.read_csv('../datasets/movies_metadata.csv')
keywords_df_raw = pd.read_csv('../datasets/keywords.csv')

C:\Users\gabri\AppData\Local\Temp\ipykernel_20376\424710010.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df_raw = pd.read_csv('../datasets/movies_metadata.csv')


In [3]:
print(f"Movies columns: {movies_df_raw.columns}")
print(f"Keywords columns: {keywords_df_raw.columns}")

Movies columns: Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')
Keywords columns: Index(['id', 'keywords'], dtype='object')


In [4]:
movies_columns = ['id', 'title', 'genres']
movies_df = movies_df_raw[movies_columns].copy()
movies_df.head()

,id,title,genres
0,862,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,8844,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,15602,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,31357,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,11862,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]"


In [5]:
keywords_df = keywords_df_raw.copy()
keywords_df['id'] = keywords_df['id'].astype(str)
keywords_df.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [6]:
movies_keywords_df = movies_df.merge(keywords_df, left_on='id', right_on='id', how='left')
movies_keywords_df.head()

,id,title,genres,keywords
0,862,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [7]:
movies_keywords_df['genres'] = movies_keywords_df['genres'].apply(lambda genre: '{}' if pd.isna(genre) else genre)
movies_keywords_df['keywords'] = movies_keywords_df['keywords'].apply(lambda keyword: '{}' if pd.isna(keyword) else keyword)

In [8]:
movies_keywords_df['genres_str'] = movies_keywords_df['genres'].apply(lambda genre: ' '.join([i['name'] for i in eval(genre)]))
movies_keywords_df['keywords_str'] = movies_keywords_df['keywords'].apply(lambda keyword: ' '.join([i['name'] for i in eval(keyword)]))

movies_keywords_df.head()

,id,title,genres,keywords,genres_str,keywords_str
0,862,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",Animation Comedy Family,jealousy toy boy friendship friends rivalry bo...
1,8844,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 10090, 'name': 'board game'}, {'id': 1...",Adventure Fantasy Family,board game disappearance based on children's b...
2,15602,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392...",Romance Comedy,fishing best friend duringcreditsstinger old men
3,31357,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[{'id': 818, 'name': 'based on novel'}, {'id':...",Comedy Drama Romance,based on novel interracial relationship single...
4,11862,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...",Comedy,baby midlife crisis confidence aging daughter ...


In [9]:
text_movies_df = movies_keywords_df[['id', 'title', 'genres_str', 'keywords_str']].copy()
text_movies_df.head()

,id,title,genres_str,keywords_str
0,862,Toy Story,Animation Comedy Family,jealousy toy boy friendship friends rivalry bo...
1,8844,Jumanji,Adventure Fantasy Family,board game disappearance based on children's b...
2,15602,Grumpier Old Men,Romance Comedy,fishing best friend duringcreditsstinger old men
3,31357,Waiting to Exhale,Comedy Drama Romance,based on novel interracial relationship single...
4,11862,Father of the Bride Part II,Comedy,baby midlife crisis confidence aging daughter ...


In [10]:
text_movies_df.loc[text_movies_df['title'].isna(), 'title'] = ''
text_movies_df['text'] = text_movies_df['title'] + ' ' + text_movies_df['genres_str'] + ' ' + text_movies_df['keywords_str']
text_movies_df.head()

,id,title,genres_str,keywords_str,text
0,862,Toy Story,Animation Comedy Family,jealousy toy boy friendship friends rivalry bo...,Toy Story Animation Comedy Family jealousy toy...
1,8844,Jumanji,Adventure Fantasy Family,board game disappearance based on children's b...,Jumanji Adventure Fantasy Family board game di...
2,15602,Grumpier Old Men,Romance Comedy,fishing best friend duringcreditsstinger old men,Grumpier Old Men Romance Comedy fishing best f...
3,31357,Waiting to Exhale,Comedy Drama Romance,based on novel interracial relationship single...,Waiting to Exhale Comedy Drama Romance based o...
4,11862,Father of the Bride Part II,Comedy,baby midlife crisis confidence aging daughter ...,Father of the Bride Part II Comedy baby midlif...


In [11]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = ' '.join(text.split())

    return text

text_movies_df['processed_text'] = text_movies_df['text'].apply(preprocess_text)
text_movies_df.head()

,id,title,genres_str,keywords_str,text,processed_text
0,862,Toy Story,Animation Comedy Family,jealousy toy boy friendship friends rivalry bo...,Toy Story Animation Comedy Family jealousy toy...,toy story animation comedy family jealousy toy...
1,8844,Jumanji,Adventure Fantasy Family,board game disappearance based on children's b...,Jumanji Adventure Fantasy Family board game di...,jumanji adventure fantasy family board game di...
2,15602,Grumpier Old Men,Romance Comedy,fishing best friend duringcreditsstinger old men,Grumpier Old Men Romance Comedy fishing best f...,grumpier old men romance comedy fishing best f...
3,31357,Waiting to Exhale,Comedy Drama Romance,based on novel interracial relationship single...,Waiting to Exhale Comedy Drama Romance based o...,waiting to exhale comedy drama romance based o...
4,11862,Father of the Bride Part II,Comedy,baby midlife crisis confidence aging daughter ...,Father of the Bride Part II Comedy baby midlif...,father of the bride part ii comedy baby midlif...


In [12]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [13]:
stop_words = set(stopwords.words('english'))
tfidf_vectorizer = TfidfVectorizer(stop_words=list(stop_words), ngram_range=(1, 2), max_features=10000)
tfidf_vectorizer_matrix = tfidf_vectorizer.fit_transform(text_movies_df['processed_text'])

feature_names = tfidf_vectorizer.get_feature_names_out()
tfidf_vectorizer_df = pd.DataFrame.sparse.from_spmatrix(tfidf_vectorizer_matrix, columns=feature_names, index=text_movies_df['id'])
tfidf_vectorizer_df.head()

,abandoned,abandonment,abbott,abbott costello,abduction,aborigine,abortion,abraham,abraham lincoln,abroad,...,zombie based,zombie violence,zombie zombie,zombies,zombies horror,zone,zone action,zoo,zorro,zulu
id,,,,,,,,,,,,,,,,,,,,,
862,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8844,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15602,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31357,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11862,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
tfidf_sim_scores = cosine_similarity(tfidf_vectorizer_matrix)

In [15]:
def get_similatiry_df(similarity_scores, movie_id):
    movie_index = text_movies_df[text_movies_df['id'] == movie_id].index[0]
    similarity = similarity_scores[movie_index]

    similarity_scores_df = pd.DataFrame(similarity, columns=['similarity'])
    return pd.concat([text_movies_df, similarity_scores_df], axis=1)

In [16]:
def find_similar_movies(movie_id, sim_scores, max_results=5):
    final_df = get_similatiry_df(sim_scores, movie_id)

    final_df_sorted = final_df.sort_values(by='similarity', ascending=False).head(max_results)
    return final_df_sorted[['id', 'title', 'similarity']]

In [17]:
find_similar_movies('862', tfidf_sim_scores)

,id,title,similarity
0,862,Toy Story,1.000000
15379,10193,Toy Story 3,0.690254
16186,56648,The Toy,0.647186
25849,256835,Toy Story That Time Forgot,0.623538
3002,863,Toy Story 2,0.542479


In [18]:
find_similar_movies('8844', tfidf_sim_scores)

,id,title,similarity
1,8844,Jumanji,1.000000
30303,262788,The Games Maker,0.515818
14313,16523,Where the Wild Things Are,0.415916
27749,30576,Karlsson on the Roof,0.399043
31363,254472,Stung,0.378044
